In [1]:
import pandas as pd
import os
import hashlib
import json

In [2]:
MTURK_DATA_PATH = r'mturk_data'
PLAYERS_DATA_NAMES = ['mturk_players_part1.csv', 'mturk_players_part2.csv']
META_DATA_NAME = 'mturk_meta_data.csv'
PRECESSED_DATA_PATH = r'processed_human_data'
CONFIG_PLAYED_NAME = 'config_played_df.csv'
NON_COMPLETE_CONFIGS_NAME = 'non_complete_configs.csv'

In [10]:
quiz_failed = 'FAILED'
quiz_failed_start = quiz_failed + '_START'
quiz_failed_end = quiz_failed + '_END'
quiz_success = 'SUCCESS'

## Preprocess

### Read the data and initial filtering

In [3]:
players_data_paths = [os.path.join(MTURK_DATA_PATH, name) for name in PLAYERS_DATA_NAMES]
meta_data_path = os.path.join(MTURK_DATA_PATH, META_DATA_NAME)

In [4]:
if type(players_data_paths) == list:
    players_df = pd.concat([pd.read_csv(path) for path in players_data_paths])
else:
    players_df = pd.read_csv(players_data_paths)
meta_df = pd.read_csv(meta_data_path)

C:\Users\irein\AppData\Local\Temp\ipykernel_9948\3000554332.py:2: DtypeWarning: Columns (15) have mixed types. Specify dtype option on import or set low_memory=False.
  players_df = pd.concat([pd.read_csv(path) for path in players_data_paths])


In [5]:
# Filter out games that are not from mturk or are demo games
print(len(meta_df))
meta_df = meta_df[meta_df['is_mturk'] == True]
print(len(meta_df))
meta_df = meta_df[meta_df['is_demo'] == False]
print(len(meta_df))

7367
7356
7356


In [6]:
# merge the game data with the metadata
meta_df.rename(columns={'id': 'participant_id'}, inplace=True)
print(players_df.shape[0])
raw_df = pd.merge(players_df, meta_df, on=['participant_id', 'session_id'])
assert raw_df.shape[0] == pd.merge(players_df, meta_df, on='participant_id').shape[0]
print(raw_df.shape[0])

184175
183900


In [7]:
assert (raw_df[(raw_df['real_turn'] != raw_df['round_number']) & ~ raw_df['real_turn'].isna()]).shape[0] == 0

In [8]:
# print the number of unique session_id and id_in_group combinations
print(len((raw_df['session_id'].astype(str) + '_' + raw_df['id_in_group'].astype(str)).value_counts()))

7356


We can group by games using the combination of the session_id and the id_in_group or just use participant_id as the unique identifier for each game.

In [9]:
relevant_columns = ['session_id', 'config_path', 'id_in_group', 'participant_id', 'player_name', 'real_turn', 'who_propose', 
                    'offer', 'proposer_message', 'proposer_recommendation', 'accepted', 'additional_info',
                    'utility', 'time_spent_on_action_page', 'show_instructions', 'quiz_answer', 
                    'mturk_worker_id', 'mturk_assignment_id']
relevant_df = raw_df[relevant_columns]
relevant_df = relevant_df.copy()

We count the number of games in which the player failed each of the quizzes. We calculate it before the "isna" filtering because players we failed in the first attempt will have a NaN values in the "config_path" column.

In [11]:
# failed statistics
relevant_df.loc[:, 'game_id'] = relevant_df['session_id'].astype(str) + '_' + relevant_df['id_in_group'].astype(str)
first_failed = len(relevant_df[relevant_df['quiz_answer'] == quiz_failed_start]['game_id'].unique())
last_failed = len(relevant_df[relevant_df['quiz_answer'] == quiz_failed_end]['game_id'].unique())
print(f'First Failed: {first_failed}')
print(f'Last Failed: {last_failed}')

First Failed: 412
Last Failed: 835


In [12]:
relevant_df.loc[:, 'game_type'] = relevant_df['config_path'].apply(lambda x: x.split('/')[0] if type(x) == str else None)
for game_type in relevant_df['game_type'].unique():
    if game_type is None:
        continue
    type_games = relevant_df[relevant_df['game_type'] == game_type]
    # first_failed = len(type_games[type_games['quiz_answer'] == quiz_failed_start]['game_id'].unique())
    last_failed = len(type_games[type_games['quiz_answer'] == quiz_failed_end]['game_id'].unique())
    # print(f'First Failed {game_type}: {first_failed}')
    print(f'Last Failed: {game_type}: {last_failed}')

Last Failed: negotiation: 268
Last Failed: persuasion: 68
Last Failed: bargaining: 499


In [13]:
print(len(relevant_df['game_id'].unique()))

7356


In [14]:
# if config_path is nan, then the row is not relevant
# otree collect rows for every possible game round but not every round includes player real actions (some are empty)
nan_rows = relevant_df['config_path'].isna()
clean_df = relevant_df[~nan_rows].copy()
print(f'Rows Number: {len(clean_df)}')

Rows Number: 17747


In [15]:
print(len((clean_df['session_id'].astype(str) + '_' + clean_df['id_in_group'].astype(str)).value_counts()))

4302


In [16]:
clean_df['game_id'] = clean_df['session_id'].astype(str) + '_' + clean_df['id_in_group'].astype(str)

In [17]:
clean_df['game_type'] = clean_df['config_path'].apply(lambda x: x.split('/')[0])
clean_df['game_type'].value_counts()

game_type
persuasion     11346
bargaining      3964
negotiation     2437
Name: count, dtype: int64

## Advanced filtering

We filter players who failed to answer the control questions correctly and filter unfinished games. The filter of the quiz automatically filters games that were not finished. 

In [18]:
print("Games:", len(clean_df['game_id'].unique()))
good_games = clean_df[clean_df['quiz_answer'] == quiz_success]['game_id']
# remove games that failed the quiz
clean_df = clean_df[clean_df['game_id'].isin(good_games)]
print(f'Rows Number: {len(clean_df)}')
clean_df.drop_duplicates(inplace=True)
print(f'Rows Number: {len(clean_df)}')
print("Games:", len(clean_df['game_id'].unique()))

Games: 4302
Rows Number: 14815
Rows Number: 14815
Games: 3405


In [19]:
bargaining_config = pd.read_csv(r'human_configs\bargaining.csv')
persuasion_config = pd.read_csv(r'human_configs\persuasion.csv')
negotiation_config = pd.read_csv(r'human_configs\negotiation.csv')

all_configs = {'bargaining' : bargaining_config, 'persuasion' : persuasion_config, 'negotiation' : negotiation_config}

In [20]:
configs_options = list(clean_df['config_path'].value_counts().index)

In [21]:
for config in configs_options:
    game_type = config.split('/')[0]
    config_id = int(config.split('/')[1])
    config_file = all_configs[game_type]
    config_data = config_file[config_file['id'] == config_id]
    assert len(config_data) >= 1, f'config {config} not found in {game_type} config file'
    assert len(config_data) == 1, f'config {config} found more than once in {game_type} config file'

In [22]:
persuasion_games = clean_df[clean_df['game_type'] == 'persuasion']
print(f'Persuasion Games rows: {len(persuasion_games)}')
print(f'Persuasion Games unique games: {len(persuasion_games["game_id"].unique())}')

Persuasion Games rows: 9700
Persuasion Games unique games: 485


In [23]:
bargaining_games = clean_df[clean_df['game_type'] == 'bargaining']
print(f'Bargaining Games rows: {len(bargaining_games)}')
print(f'Bargaining Games unique games: {len(bargaining_games["game_id"].unique())}')

Bargaining Games rows: 3100
Bargaining Games unique games: 1696


In [24]:
negotiation_games = clean_df[clean_df['game_type'] == 'negotiation']
print(f'Negotiation Games rows: {len(negotiation_games)}')
print(f'Negotiation Games unique games: {len(negotiation_games["game_id"].unique())}')

Negotiation Games rows: 2015
Negotiation Games unique games: 1224


## Configuration count

In [25]:
# configuration counts
configs_played_duplicates = clean_df.groupby('game_id')['config_path'].max()
configs_played_duplicates.sort_values()
config_played = configs_played_duplicates.value_counts()

In [26]:
# config_played to dataframe
config_played_df = config_played.reset_index()
config_played_df['game_type'] = config_played_df['config_path'].apply(lambda x: x.split('/')[0])
config_played_df['config_id'] = config_played_df['config_path'].apply(lambda x: int(x.split('/')[1]))
config_played_df = config_played_df.sort_values(by=['game_type', 'config_id'])
config_played_df = config_played_df.reset_index(drop=True)
config_played_df = config_played_df[['game_type', 'config_id', 'config_path', 'count']]
config_played_df.rename(columns={'count': 'games_played'}, inplace=True)

In [27]:
def find_games_required(game_type, config_id):
    config_file = all_configs[game_type]
    config = config_file[config_file['id'] == config_id]
    assert len(config) == 1, f'config {config_id} not found in {game_type} config file'
    return config['games'].values[0]

def find_qualification(game_type, config_id):
    config_file = all_configs[game_type]
    config = config_file[config_file['id'] == config_id]
    assert len(config) == 1, f'config {config_id} not found in {game_type} config file'
    return config['qualification'].values[0]

In [28]:
config_played_df['games_required'] = \
    config_played_df.apply(lambda x: find_games_required(x['game_type'], x['config_id']), axis=1)
config_played_df['game_diff'] = config_played_df['games_played'] - config_played_df['games_required']
config_played_df['qualification'] = \
    config_played_df.apply(lambda x: find_qualification(x['game_type'], x['config_id']), axis=1)

In [29]:
all_configs_ids = dict()
for game_type, config_file in all_configs.items():
    for index, row in config_file.iterrows():
        temp_id = game_type + '/' + str(row['id'])
        all_configs_ids[temp_id] = (row['games'], row['qualification'])

In [30]:
config_id_played = config_played_df['config_path'].values
non_played_configs = dict()
for config_id in all_configs_ids.keys():
    if config_id not in config_id_played:
        non_played_configs[config_id] = all_configs_ids[config_id]
        new_row = {'game_type': config_id.split('/')[0],
                   'config_id': int(config_id.split('/')[1]),
                   'config_path': config_id,
                   'games_played': 0,
                   'games_required': all_configs_ids[config_id][0],
                   'game_diff': -all_configs_ids[config_id][0],
                   'qualification': all_configs_ids[config_id][1]}
        config_played_df = pd.concat([config_played_df, pd.DataFrame([new_row])])

In [31]:
non_played_configs

{}

In [32]:
config_played_df['game_diff'].value_counts().sort_index()

game_diff
0     76
1    114
2      4
3      1
Name: count, dtype: int64

In [33]:
negatives = config_played_df[config_played_df['game_diff'] < 0]
print("Completion games needed: ", -1 * negatives['game_diff'].sum())

Completion games needed:  0


In [34]:
config_played_df_path = os.path.join(MTURK_DATA_PATH, CONFIG_PLAYED_NAME)
config_played_df.to_csv(config_played_df_path, index=False)

In [35]:
non_complete_config = config_played_df[config_played_df['game_diff'] < 0].copy()
non_complete_config['amount'] = (-1 * non_complete_config['game_diff']) + 1
non_complete_config = non_complete_config[['game_type', 'qualification', 'config_id', 'amount',
                                           'games_played', 'games_required', 'game_diff']]
non_complete_config.sort_values(by=['game_type', 'qualification', 'config_id'], inplace=True)

In [36]:
non_complete_config

,game_type,qualification,config_id,amount,games_played,games_required,game_diff


In [37]:
non_complete_config_path = os.path.join(MTURK_DATA_PATH, NON_COMPLETE_CONFIGS_NAME)
non_complete_config.to_csv(non_complete_config_path, index=False)

## Transform to DataLogger format - Bargaining Game

In [38]:
def find_max_offer_by_config_id(config_id):
    config_row = bargaining_config[bargaining_config['id'] == config_id]
    assert len(config_row) == 1, f'config {config_id} not found in bargaining config file: ' + str(len(config_row))
    return config_row['game_args_money_to_divide'].values[0]

def find_human_is_player_by_config_id(config_id):
    config_row = bargaining_config[bargaining_config['id'] == config_id]
    assert len(config_row) == 1, f'config {config_id} not found in bargaining config file: ' + str(len(config_row))
    return config_row['human_is_player'].values[0]

In [39]:
os.makedirs(f'{PRECESSED_DATA_PATH}/bargaining', exist_ok=True)
rel_bargaining = bargaining_games[['config_path', 'offer', 'proposer_message', 'who_propose', 'accepted', 
                                   'game_id', 'player_name', 'real_turn']]
games_ids = rel_bargaining['game_id'].unique()

In [40]:
for cur_game_id in games_ids:
    cur_game = rel_bargaining[rel_bargaining['game_id'] == cur_game_id].sort_values('real_turn')
    config_id = int(cur_game['config_path'].values[0].split('/')[1])
    assert cur_game['config_path'].nunique() == 1, f'game {cur_game_id} has more than one config'
    row_columns = [f'alice_gain', 'bob_gain', 'message', 'player', 'round', 'decision']
    columns_types = {'player': str, 'round': int, 'decision': str, 'message': str, f'alice_gain': float, 'bob_gain': float}
    human_is_alice = find_human_is_player_by_config_id(config_id) == 1
    tmp_df = None
    for i, row in cur_game.iterrows():
        who_offer = row['who_propose']
        proposer_gain = find_max_offer_by_config_id(config_id) - row['offer']
        receiver_gain = row['offer']
        message = row['proposer_message']
        round_number = int(row['real_turn'])
        player_name = 'Alice' if round_number % 2 == 1 else 'Bob'
        new_row = pd.DataFrame([[proposer_gain, receiver_gain, message, player_name, round_number, None]], columns=row_columns)
        new_row = new_row.astype(columns_types)
        if tmp_df is None:
            tmp_df = new_row
        else:
            tmp_df = pd.concat([tmp_df, new_row], ignore_index=True)
        
        other_player_name = 'Bob' if player_name == 'Alice' else 'Alice'
        accepted = 'accept' if row['accepted'] else 'reject'
        new_row = pd.DataFrame([[None, None, None, other_player_name, round_number, accepted]], columns=row_columns)
        new_row = new_row.astype(columns_types)
        tmp_df = pd.concat([tmp_df, new_row], ignore_index=True)
        
    hash_game_id = cur_game_id.encode('utf-8')
    hash_object = hashlib.sha256()
    hash_object.update(hash_game_id)
    hash_game_id = hash_object.hexdigest()
    hash_game_id = (str(hash_game_id)[:16]).upper()
    
    dir_name = f'{PRECESSED_DATA_PATH}/bargaining/{hash_game_id[0]}/{hash_game_id[1]}/{hash_game_id[2]}'
    os.makedirs(rf'{dir_name}/{hash_game_id}', exist_ok=True)
    
    tmp_df.to_csv(rf'{dir_name}/{hash_game_id}/game.csv', index=False)
    source_path = f'configs/bargaining/{config_id}.json'
    with open(source_path, 'r') as f:
        config_data = json.load(f)
        config_data['player_1_args']['model_name'] = config_data['player_1_type']
        config_data['player_2_args']['model_name'] = config_data['player_2_type']
        with open(rf'{dir_name}/{hash_game_id}/config.json', 'w') as f2:
            json.dump(config_data, f2, indent=4)
            f2.write('\n')

## Transform to DataLogger format - Persuasion Game

In [41]:
def find_value_by_config_id(config_id, is_high_quality):
    config_row = persuasion_config[persuasion_config['id'] == config_id]
    assert len(config_row) == 1, f'config {config_id} not found in persuasion config file: ' + str(len(config_row))
    game_args_c = config_row['game_args_c'].values[0]
    game_args_v = config_row['game_args_v'].values[0]
    game_args_product_price = config_row['game_args_product_price'].values[0]
    if is_high_quality:
        return game_args_v * game_args_product_price
    else:
        return game_args_c * game_args_product_price
    
def find_seller_message_type_by_config_id(config_id):
    config_row = persuasion_config[persuasion_config['id'] == config_id]
    assert len(config_row) == 1, f'config {config_id} not found in persuasion config file: ' + str(len(config_row))
    return config_row['game_args_seller_message_type'].values[0]

In [42]:
os.makedirs(f'{PRECESSED_DATA_PATH}/persuasion', exist_ok=True)
rel_persuasion = persuasion_games[['config_path', 'proposer_recommendation', 'proposer_message', 'accepted', 
                                   'game_id', 'player_name', 'real_turn', 'additional_info']]
games_ids = rel_persuasion['game_id'].unique()

seller_name = 'Alice'
buyer_name = 'Bob'
nature_name = 'Nature'

In [43]:
for cur_game_id in games_ids:
    cur_game = rel_persuasion[rel_persuasion['game_id'] == cur_game_id].sort_values('real_turn')
    config_id = int(cur_game['config_path'].values[0].split('/')[1])
    assert cur_game['config_path'].nunique() == 1, f'game {cur_game_id} has more than one config'
    message_type = find_seller_message_type_by_config_id(config_id)
    row_columns = [f'round_quality', 'product_worth', 'player', 'round', 'message', 'decision']
    tmp_df = None
    for i, row in cur_game.iterrows():
        quality = 'high-quality' if row['additional_info'] else 'low-quality'
        product_worth = find_value_by_config_id(config_id, row['additional_info'])
        round_number = int(row['real_turn'])
        new_row = pd.DataFrame([[quality, product_worth, nature_name, round_number, None, None]], columns=row_columns)
        if tmp_df is None:
            tmp_df = new_row
        else:
            tmp_df = pd.concat([tmp_df, new_row], ignore_index=True)
            
        if message_type == 'text':
            message = row['proposer_message']
            assert row['proposer_recommendation'] is not None, f'row {i} has None proposer_recommendation'
            new_row = pd.DataFrame([[None, None, seller_name, round_number, message, None]], columns=row_columns)
        else:  # binary
            assert row['proposer_recommendation'] is not None, f'row {i} has None proposer_recommendation'
            decision = 'yes' if row['proposer_recommendation'] else 'no'
            new_row = pd.DataFrame([[None, None, seller_name, round_number, None, decision]], columns=row_columns)
        
        tmp_df = pd.concat([tmp_df, new_row], ignore_index=True)
        
        decision = 'yes' if row['accepted'] else 'no'
        new_row = pd.DataFrame([[None, None, buyer_name, round_number, None, decision]], columns=row_columns)
        tmp_df = pd.concat([tmp_df, new_row], ignore_index=True)
        
    hash_game_id = cur_game_id.encode('utf-8')
    hash_object = hashlib.sha256()
    hash_object.update(hash_game_id)
    hash_game_id = hash_object.hexdigest()
    hash_game_id = (str(hash_game_id)[:16]).upper()
    dir_name = f'{PRECESSED_DATA_PATH}/persuasion/{hash_game_id[0]}/{hash_game_id[1]}/{hash_game_id[2]}'
    os.makedirs(rf'{dir_name}/{hash_game_id}', exist_ok=True)
    tmp_df.to_csv(rf'{dir_name}/{hash_game_id}/game.csv', index=False)
    source_path = f'configs/persuasion/{config_id}.json'
    with open(source_path, 'r') as f:
        config_data = json.load(f)
        config_data['player_1_args']['model_name'] = config_data['player_1_type']
        config_data['player_2_args']['model_name'] = config_data['player_2_type']
        with open(rf'{dir_name}/{hash_game_id}/config.json', 'w') as f2:
            json.dump(config_data, f2, indent=4)
            f2.write('\n')
            

C:\Users\irein\AppData\Local\Temp\ipykernel_9948\2359822592.py:27: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  tmp_df = pd.concat([tmp_df, new_row], ignore_index=True)
C:\Users\irein\AppData\Local\Temp\ipykernel_9948\2359822592.py:31: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  tmp_df = pd.concat([tmp_df, new_row], ignore_index=True)


## Transform to DataLogger format - Negotiation Game

In [44]:
os.makedirs(f'{PRECESSED_DATA_PATH}/negotiation', exist_ok=True)
rel_negotiation = negotiation_games[['config_path', 'offer', 'proposer_message', 'accepted', 
                                   'game_id', 'player_name', 'real_turn']]
games_ids = rel_negotiation['game_id'].unique()

seller_name = 'Alice'
buyer_name = 'Bob'

In [45]:
for cur_game_id in games_ids:
    cur_game = rel_negotiation[rel_negotiation['game_id'] == cur_game_id].sort_values('real_turn')
    config_id = int(cur_game['config_path'].values[0].split('/')[1])
    assert cur_game['config_path'].nunique() == 1, f'game {cur_game_id} has more than one config'
    
    row_columns = ['message', 'product_price', 'player', 'round', 'decision']
    tmp_df = None
    for i, row in cur_game.iterrows():
        product_price = row['offer']
        round_number = int(row['real_turn'])
        player_name = 'Alice' if round_number % 2 == 1 else 'Bob'
        message = row['proposer_message']
        new_row = pd.DataFrame([[message, product_price, player_name, round_number, None]], columns=row_columns)
        
        if tmp_df is None:
            tmp_df = new_row
        else:
            tmp_df = pd.concat([tmp_df, new_row], ignore_index=True)
            
        other_player_name = 'Bob' if player_name == 'Alice' else 'Alice'
        accepted = row['accepted']
        if accepted == 2:
            decision = 'AcceptOffer'
        elif accepted == 1:
            decision = 'RejectOffer'
        else:  # accepted == 0
            decision = 'BuyFromJhon' if other_player_name == 'Bob' else 'SellToJhon'
        new_row = pd.DataFrame([[None, None, other_player_name, round_number, decision]], columns=row_columns)
        tmp_df = pd.concat([tmp_df, new_row], ignore_index=True)
        
        
    hash_game_id = cur_game_id.encode('utf-8')
    hash_object = hashlib.sha256()
    hash_object.update(hash_game_id)
    hash_game_id = hash_object.hexdigest()
    hash_game_id = (str(hash_game_id)[:16]).upper()
    dir_name = f'{PRECESSED_DATA_PATH}/negotiation/{hash_game_id[0]}/{hash_game_id[1]}/{hash_game_id[2]}'
    os.makedirs(rf'{dir_name}/{hash_game_id}', exist_ok=True)
    tmp_df.to_csv(rf'{dir_name}/{hash_game_id}/game.csv', index=False)
    source_path = f'configs/negotiation/{config_id}.json'
    with open(source_path, 'r') as f:
        config_data = json.load(f)
        config_data['player_1_args']['model_name'] = config_data['player_1_type']
        config_data['player_2_args']['model_name'] = config_data['player_2_type']
        with open(rf'{dir_name}/{hash_game_id}/config.json', 'w') as f2:
            json.dump(config_data, f2, indent=4)
            f2.write('\n')

C:\Users\irein\AppData\Local\Temp\ipykernel_9948\1969595530.py:29: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  tmp_df = pd.concat([tmp_df, new_row], ignore_index=True)
